In [1]:
from langchain_groq import ChatGroq 

In [2]:
# !pip install langchain_groq

In [8]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import WebBaseLoader
import re

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
llm = ChatGroq(
    temperature = 0,
    model_name = "deepseek-r1-distill-qwen-32b",
    groq_api_key='api_key'
)
response = llm.invoke("current prime minister of india is ..")
print(response.content)

<think>
Okay, so I need to figure out who the current Prime Minister of India is. I'm not entirely sure, but I think it's someone who's been in the news a lot recently. Let me try to recall. I remember hearing about a Prime Minister who has been in office for a few years now. His name starts with an "N," I believe. Wasn't there a big election a while back where he was re-elected? 

Wait, I think his name is Narendra Modi. Yeah, that sounds right. He's the leader of the Bharatiya Janata Party, if I'm not mistaken. I remember seeing his face a lot on the news, especially during important events or speeches. He's been quite active in promoting various policies and initiatives in India. 

I should double-check to make sure I'm not confusing him with someone else. Maybe I can think of other Prime Ministers India has had recently. There was Manmohan Singh before Modi, right? And before that, Atal Bihari Vajpayee. So, Modi must be the current one. 

I also recall that Modi was the Chief Minis

In [3]:
#webbaseloader - accept url of website nad extract data
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.ndtv.com/india-news/5-cheetahs-released-into-wild-in-madhya-pradeshs-kuno-national-park-7765686#pfrom=home-ndtv_lateststories")
page_data= loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


 


5 Cheetahs Released Into Wild In Madhya Pradesh's Kuno National Park










































































































































arrow-down
















comments
















printer























search





bell







top-nav



















































right-arrow














































































































NDTV 


World 


Profit 


हिंदी 


WPL 2025 


MOVIES 


FOOD 


LIFESTYLE 


HEALTH 


SWASTH 


TECH 


Games 


Big Bonus 


SHOPPING 


APPS 














Advertisement





































Live TV
        







































































































































































































Latest 
India  






 

India
All India News India GlobalDiasporaSouth 


In [4]:
import re
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<[^>]*?>', '', text)
    # Remove URLs
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s{2,}', ' ', text)
    # Trim leading and trailing whitespace
    text = text.strip()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

In [5]:
type(page_data)

str

In [6]:
clean_text(page_data)

'5 Cheetahs Released Into Wild In Madhya Pradeshs Kuno National ParkarrowdowncommentsprintersearchbelltopnavrightarrowNDTV World Profit WPL 2025 MOVIES FOOD LIFESTYLE HEALTH SWASTH TECH Games Big Bonus SHOPPING APPS AdvertisementLive TV Latest India IndiaAll India News India GlobalDiasporaSouth Latest Congress Leaders Sleepover At Rajasthan Assembly After Dadi Remark Row Elder Brother Mentor Bhutan PM Praises PM Modis Leadership Want To Be Fair Trump Vows To Impose Reciprocal Tariffs On India Will Impose Retaliatory Tariffs For Digital Taxes May Come Friday Trump Swami Ramdev Invites Bryan Johnson To Haridwar After Air Quality ConcernsWorld Cities MetrosBengaluruChennaiDelhiMumbaiOther CitiesAhmedabadAllahabadAmritsarBhopalBhubaneshwarChandigarhGhaziabadGoaGurgaonGuwahatiHyderabadJaipurKanpurKolkataLucknowNoidaPatnaPuneSrinagarThiruvananthapuramCT 2025 Opinion Videos IndiaAI Auto Education EducationExamsSchool CampusStudy AbroadWeb Stories Latest CBSE To Conduct Class 10 Sanskrit Class

In [9]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
jres = json_parser.parse(res.content)
jres

{'Type': 'Hard News',
 'Subtype': 'Environmental News',
 'News Headline': "5 Cheetahs Released Into Wild In Madhya Pradesh's Kuno National Park",
 'Region': 'India',
 'State': 'Madhya Pradesh',
 'Country': 'India',
 'description': 'Five cheetahs, including Jwala and her four cubs, were released into the wild at Kuno National Park in Madhya Pradesh. This brings the total number of cheetahs in the wild to 12, with 14 still in enclosures. The park now houses 26 cheetahs, including those imported from Namibia and South Africa, as well as locally born cubs. This initiative is part of a significant conservation effort to reintroduce cheetahs into India.'}

In [8]:
type(res.content)

str

In [9]:

def parse_and_classify_news(url):
    loader = WebBaseLoader(url)
    page_data= loader.load().pop().page_content
    clean_text(page_data)

    prompt_extract = PromptTemplate.from_template(  """
    ###SCRAPED TEXT WEBSITE:
    {page_data}
    ###INSTRUCTION:
    The scraped text is from the new's page of a official newswebsite.
    first read and understand the content and classify them under the following types and subtypes.
    here is the list to help you , make sure to classify news among them.
    type:Hard News
    subtype:
    Breaking News
    Political News
    Business News
    Crime News
    International News
    Health News
    Environmental News
    Sports News

    type:Soft News
    subtype:
    Human Interest Stories
    Entertainment News
    Lifestyle News
    Technology News
    Science News
    Cultural News
    Food News
    Education News
    Feature News

    type:Analysis
    subtype:
    Interviews
    Investigative Journalism
    Opinion Pieces/Editorials
    Reviews
    Business & Financial News

    type:Stock Market News
    subtype:
    Economy News
    Startups and Entrepreneurship News
    Technology & Digital News

    type:Techincal aspects
    subtype
    Cybersecurity News
    Social Media News
    Launch news 

    type:Local News
    subtype:
    Regional/Local Events
    Local Politics
    Local Crime

    type:Disaster and Crisis News
    subtype:
    Natural Disasters
    Humanitarian Crises
    Pandemics/Health Crises

    type:Weather News
    subtype:
    Forecasting
    Severe Weather Alerts

    type:Political Opinion & Commentary
    subtype:
    Political Analysis
    Political Debates

    type:Celebrity & Gossip News
    subtype:
    Celebrity Gossip
    Red Carpet Events

    type:Social & Public Issues News
    subtypes:
    Social Justice
    Human Rights
    Immigration
    Opinion & Editorials
    Columnists
    Letters to the Editor

    Your job is to extract headline of the news ,the type and subtype  of news among the mentioned above and find region, state and country of the news and summarise it to a description which fits in short blogs and return them in JSON format containing 
    following keys: 'Type' ,'Subtype'
    'News Headline',
    'Region','State', 'Country' and 'description'.
    Only return the valid JSON.
    ###VALID JSON (NO PREAMBLE):
    """)
    
    chain_extract = prompt_extract | llm
    res = chain_extract.invoke(input={'page_data': page_data})
    
    json_parser = JsonOutputParser()
    jres = json_parser.parse(res.content)
    return jres

In [10]:
# Example usage
news_json = parse_and_classify_news("https://www.theguardian.com/world/2024/sep/16/barnier-fights-to-form-french-government-amid-no-confidence-threats")
print(news_json)

{'Type': 'Hard News', 'Subtype': 'Political News', 'News Headline': 'Barnier fights to form French government amid no-confidence threats', 'Region': 'Europe', 'State': '', 'Country': 'France', 'description': "Michel Barnier, the newly appointed French Prime Minister, is struggling to form a government amidst threats of no-confidence votes from far-right and hard-left parties. Barnier, a veteran politician and former EU Brexit negotiator, has promised to form a new administration by the end of the week after listening to various political factions. However, the political deadlock in France, resulting from Emmanuel Macron's snap general election, has created a complex equation for Barnier to solve. The National Rally (RN), a far-right party, holds significant influence, while the left-wing coalition, the New Popular Front (NFP), is also a major player. Barnier's ability to navigate these political challenges will determine whether he can avoid a no-confidence vote and stabilize the gover

In [15]:
type(news_json)

dict

In [16]:
import pandas as pd, numpy as np

In [11]:
import chromadb

chroma_client = chromadb.PersistentClient(path="./chroma_news_db")
collection = chroma_client.get_or_create_collection(name="news_articles")

def store_news(news_json):
    collection.add(
        ids=[news_json["News Headline"]],
        documents=[news_json["description"]],
        metadatas={
            "type": news_json["Type"],
            "subtype": news_json["Subtype"],
            "region": news_json["Region"],
            "state": news_json["State"],
            "country": news_json["Country"]
        }
    )
    print("News stored in ChromaDB successfully.")
store_news(jres)

News stored in ChromaDB successfully.


In [17]:
store_news(news_json)

News stored in ChromaDB successfully.


In [18]:
response = collection.query(query_texts=["Madhya Pradesh News", "American news"],n_results =2).get('metadatas',[])
response

[[{'country': 'India',
   'region': 'India',
   'state': 'Madhya Pradesh',
   'subtype': 'Environmental News',
   'type': 'Hard News'},
  {'country': 'India',
   'region': 'India',
   'state': 'Rajasthan',
   'subtype': 'Political News',
   'type': 'Hard News'}],
 [{'country': 'India',
   'region': 'India',
   'state': 'Rajasthan',
   'subtype': 'Political News',
   'type': 'Hard News'},
  {'country': 'India',
   'region': 'India',
   'state': 'Madhya Pradesh',
   'subtype': 'Environmental News',
   'type': 'Hard News'}]]

In [21]:
job = news_json
job['Type']

'Hard News'

In [22]:
links = collection.query(query_texts=job['Type'],n_results =2).get('metadatas',[])
links

[[{'country': 'India',
   'region': 'India',
   'state': 'Madhya Pradesh',
   'subtype': 'Environmental News',
   'type': 'Hard News'},
  {'country': 'India',
   'region': 'India',
   'state': 'Rajasthan',
   'subtype': 'Political News',
   'type': 'Hard News'}]]

In [24]:
# !uvicorn filename:app --host 0.0.0.0 --port 8000 --reload

^C


In [25]:
# from fastapi import FastAPI

# app = FastAPI()
# chroma_client = chromadb.PersistentClient(path="./chroma_news_db")
# collection = chroma_client.get_collection(name="news_articles")

# # API to fetch all news
# @app.get("/news/")
# def get_news():
#     results = collection.get(include=["metadatas", "documents"])
#     return results

# # API to fetch specific category news
# @app.get("/news/{news_type}")
# def get_news_by_category(news_type: str):
#     results = collection.get(
#         where={"type": news_type}, include=["metadatas", "documents"]
#     )
#     return results

In [50]:
pipeline = RunnableLambda(parse_and_classify_news) | RunnableLambda(store_news)

In [ ]:
news_url = "https://www.theguardian.com/world/2024/sep/16/barnier-fights-to-form-french-government-amid-no-confidence-threats"
result = pipeline.invoke(news_url)
print(result)  # Output: "Stored Successfully"